# **Creating a CNN Model and Optimizing it using Keras Tuner**

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

Importing Data

In [21]:
fashion_mnist = keras.datasets.fashion_mnist
#fashion_mnist is an Image Dataset

Data Preprocessing

In [22]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

In [23]:
# Scaling down images between 0 to 1 because all the images are in gray scale format:
train_images=train_images/255.0
test_images=test_images/255.0

In [24]:
#Observing the shape of first image
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [25]:
train_images[0].shape

(28, 28)

In [6]:
#Reshaping Images
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

Building Hyperparameter Model

In [7]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [8]:
pip install keras-tuner

     |████████████████████████████████| 97 kB 3.8 MB/s 


Creating Model

In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [10]:
tuner_search=RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name="Mnist Fashion")

In [11]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 50s]
val_accuracy: 0.9101666808128357

Best val_accuracy So Far: 0.9138333201408386
Total elapsed time: 00h 05m 17s
INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        41520     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1769536   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,812,666
Trainable params: 1,812,666
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1364 - accuracy: 0.9493 - val_loss: 0.2681 - val_accuracy: 0.9107
Epoch 5/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0975 - accuracy: 0.9640 - val_loss: 0.2793 - val_accuracy: 0.9132
Epoch 6/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0688 - accuracy: 0.9755 - val_loss: 0.3121 - val_accuracy: 0.9077
Epoch 7/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0496 - accuracy: 0.9813 - val_loss: 0.3879 - val_accuracy: 0.9072
Epoch 8/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0361 - accuracy: 0.9872 - val_loss: 0.4564 - val_accuracy: 0.9075
Epoch 9/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0309 - accuracy: 0.9888 - val_loss: 0.4960 - val_accuracy: 0.9068
Epoch 10/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0247 - accuracy: 0.9913 - val_loss: 0.5464 - val_a